In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


In [ ]:
df3 = pd.read_parquet("/history.parquet")
df2 = df3.drop(["country","country_alpha2","capital"], axis = 1)
df = df2.dropna()

df['date'] = pd.to_datetime(df['date'])

df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofyear'] = df['date'].dt.dayofyear

df['sin_doy'] = np.sin(2*np.pi*df['dayofyear']/365)
df['cos_doy'] = np.cos(2*np.pi*df['dayofyear']/365)

df = df.drop(columns=['date'])

target_cols = [
    'temp_min_c','temp_max_c','temp_mean_c_approx',
    'app_temp_min_c','app_temp_max_c',
    'precip_mm','rain_mm','snow_mm',
    'windspeed_10m_max_kmh','windgusts_10m_max_kmh',
    'wind_dir_dom_deg','sunshine_duration_s',
    'daylight_duration_s','shortwave_radiation_MJ_m2'
]

X = df[['lat', 'lon', 'sin_doy', 'cos_doy']]
Y = df[target_cols]


/tmp/ipython-input-2074647786.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])
/tmp/ipython-input-2074647786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['date'].dt.month
/tmp/ipython-input-2074647786.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    objective="reg:squarederror"
)

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
def date_to_cyclical(month, day):
    # create a dummy year (year doesn't matter)
    date = pd.to_datetime(f"2024-{month}-{day}")
    doy = date.timetuple().tm_yday
    sin_doy = np.sin(2*np.pi * doy / 365)
    cos_doy = np.cos(2*np.pi * doy / 365)
    return sin_doy, cos_doy

In [ ]:
lat = 37.7749
lon = -122.4194
month = 3
day = 18

sin_doy, cos_doy = date_to_cyclical(month, day)

X_new = pd.DataFrame([{
    "lat": lat,
    "lon": lon,
    "sin_doy": sin_doy,
    "cos_doy": cos_doy
}])

pred = model.predict(X_new)

print(pred)

[[ 1.1579049e+01  1.8319750e+01  1.5367237e+01  6.3470292e+00
   1.6247181e+01  1.0903603e+00  7.3685163e-01 -2.4898882e-05
   2.8677523e+01  4.8937389e+01  2.2354915e+02  3.0358299e+04
   4.3583156e+04  1.8273447e+01]]


In [ ]:
def predict_weather(lat, lon, date_str):
    # date_str format: "MM-DD"
    month, day = map(int, date_str.split("-"))

    # convert date to cyclical features
    sin_doy, cos_doy = date_to_cyclical(month, day)

    # create input dataframe
    X_new = pd.DataFrame([{
        "lat": lat,
        "lon": lon,
        "sin_doy": sin_doy,
        "cos_doy": cos_doy
    }])

    # model must already be trained before calling this!
    preds = model.predict(X_new)[0]

    # attach names to predictions
    return pd.Series(preds, index=target_cols)

In [ ]:
result = predict_weather(37.7749, -122.4194, "03-18")
print(result)

temp_min_c                      11.579049
temp_max_c                      18.319750
temp_mean_c_approx              15.367237
app_temp_min_c                   6.347029
app_temp_max_c                  16.247181
precip_mm                        1.090360
rain_mm                          0.736852
snow_mm                         -0.000025
windspeed_10m_max_kmh           28.677523
windgusts_10m_max_kmh           48.937389
wind_dir_dom_deg               223.549149
sunshine_duration_s          30358.298828
daylight_duration_s          43583.156250
shortwave_radiation_MJ_m2       18.273447
dtype: float32
